In [ ]:
from agentex import Agentex

client = Agentex(base_url="http://localhost:5003")

In [ ]:
AGENT_NAME = "ab080-batch-events"

In [ ]:
# (REQUIRED) Create a new task. For Agentic agents, you must create a task for messages to be associated with.
import uuid

rpc_response = client.agents.create_task(
    agent_name=AGENT_NAME,
    params={
        "name": f"{str(uuid.uuid4())[:8]}-task",
        "params": {}
    }
)

task = rpc_response.result
print(task)

In [ ]:
# Send an event to the agent
from agentex.types import Event
from agentex.types.agent_rpc_params import ParamsSendEventRequest

# The response is expected to be a list of TaskMessage objects, which is a union of the following types:
# - TextContent: A message with just text content   
# - DataContent: A message with JSON-serializable data content
# - ToolRequestContent: A message with a tool request, which contains a JSON-serializable request to call a tool
# - ToolResponseContent: A message with a tool response, which contains response object from a tool call in its content

# When processing the message/send response, if you are expecting more than TextContent, such as DataContent, ToolRequestContent, or ToolResponseContent, you can process them as well

concurrent_event_messages: list[ParamsSendEventRequest] = [
    {
        "content": {"type": "text", "author": "user", "content": "Hello, what can you do?"},
        "task_id": task.id,
    },
    {
        "content": {"type": "text", "author": "user", "content": "Can you tell me a joke?"},
        "task_id": task.id,
    },
    {
        "content": {"type": "text", "author": "user", "content": "What is the capital of France?"},
        "task_id": task.id,
    },
    {
        "content": {"type": "text", "author": "user", "content": "Write a short story about a cat"},
        "task_id": task.id,
    },
    {
        "content": {"type": "text", "author": "user", "content": "Tell me how an LLM works"},
        "task_id": task.id,
    },
]

events: list[Event] = []

for event_message in concurrent_event_messages:
    rpc_response = client.agents.send_event(
        agent_name=AGENT_NAME,
        params={
            "content": {"type": "text", "author": "user", "content": "Hello tell me the latest news about AI and AI startups"},
            "task_id": task.id,
        }
    )

    event = rpc_response.result
    events.append(event)

for event in events:
    print(event)

In [ ]:
from agentex.lib.utils.dev_tools import subscribe_to_async_task_messages

task_messages = subscribe_to_async_task_messages(
    client=client,
    task=task, 
    only_after_timestamp=event.created_at, 
    print_messages=True,
    rich_print=True,
    timeout=20,
)